In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from sklearn.model_selection import cross_val_score
from keras.layers import Conv1D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from keras.layers.core import Dense, Dropout
from keras.utils import np_utils
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [25]:
path = r'E:/Data Sets'
df = pd.read_csv(os.path.join(path, 'Task-2.2-revised-data.csv'))
df.drop(columns=['Unnamed: 0'],inplace=True)
df1 = pd.read_csv(os.path.join(path, 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))
df1.drop(columns=['DATE'],inplace=True)
df = np.array(df)
df = df.reshape(-1,15,9)
df1 = np.array(df1)
df1 = df1.reshape(22950,-1)
df1 = np.argmax(df1, axis = 1)
X_train, X_test, y_train, y_test = train_test_split(df,df1,random_state = 42)

In [26]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


In [27]:
df1

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [28]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y_train)

'multiclass'

In [18]:
type_of_target(Y_train)

'multiclass'

In [32]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [41]:
def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'SGD']
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]  #optimizerD[optimizerL[round(optimizer)]]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [42]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), #9
    'optimizer':(0,7), #7
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), #(10, 50), #
    'epochs':(20, 100),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=10, n_iter=3) #25
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/32
 - 2s - loss: 2.4984 - accuracy: 0.3603
Epoch 2/32
 - 2s - loss: 1.8553 - accuracy: 0.6433
Epoch 3/32
 - 2s - loss: 1.5393 - accuracy: 0.6440
Epoch 4/32
 - 2s - loss: 1.3971 - accuracy: 0.6440
Epoch 5/32
 - 2s - loss: 1.3126 - accuracy: 0.6440
Epoch 6/32
 - 2s - loss: 1.2587 - accuracy: 0.6440
Epoch 7/32
 - 2s - loss: 1.2214 - accuracy: 0.6440
Epoch 8/32
 - 2s - loss: 1.1947 - accuracy: 0.6440
Epoch 9/32
 - 2s - loss: 1.1728 - accuracy: 0.6440
Epoch 10/32
 - 2s - loss: 1.1571 - accuracy: 0.6440
Epoch 11/32
 - 2s - loss: 1.1437 - accuracy: 0.6440
Epoch 12/32
 - 2s - loss: 1.1325 - accuracy: 0.6440
Epoch 13/32
 - 2s - loss: 1.1236 - accuracy: 

Epoch 25/32
 - 2s - loss: 1.0547 - accuracy: 0.6439
Epoch 26/32
 - 2s - loss: 1.0518 - accuracy: 0.6473
Epoch 27/32
 - 2s - loss: 1.0494 - accuracy: 0.6471
Epoch 28/32
 - 2s - loss: 1.0467 - accuracy: 0.6463
Epoch 29/32
 - 2s - loss: 1.0451 - accuracy: 0.6468
Epoch 30/32
 - 2s - loss: 1.0420 - accuracy: 0.6479
Epoch 31/32
 - 2s - loss: 1.0400 - accuracy: 0.6497
Epoch 32/32
 - 2s - loss: 1.0381 - accuracy: 0.6479
| 1         | 0.6484    | 3.371     | 960.6     | 0.732     | 0.1796    | 32.48     | 1.312     | 1.116     | 2.732     | 0.6051    | 73.73     | 0.02058   | 6.789     |
Epoch 1/44
 - 1s - loss: nan - accuracy: 0.6440
Epoch 2/44
 - 1s - loss: nan - accuracy: 0.6440
Epoch 3/44
 - 1s - loss: nan - accuracy: 0.6440
Epoch 4/44
 - 1s - loss: nan - accuracy: 0.6440
Epoch 5/44
 - 1s - loss: nan - accuracy: 0.6440
Epoch 6/44
 - 1s - loss: nan - accuracy: 0.6440
Epoch 7/44
 - 1s - loss: nan - accuracy: 0.6440
Epoch 8/44
 - 1s - loss: nan - accuracy: 0.6440
Epoch 9/44
 - 1s - loss: nan -

 - 3s - loss: 0.3812 - accuracy: 0.8603
Epoch 35/67
 - 3s - loss: 0.3608 - accuracy: 0.8667
Epoch 36/67
 - 3s - loss: 0.3516 - accuracy: 0.8690
Epoch 37/67
 - 3s - loss: 0.3444 - accuracy: 0.8739
Epoch 38/67
 - 3s - loss: 0.3485 - accuracy: 0.8713
Epoch 39/67
 - 3s - loss: 0.3527 - accuracy: 0.8708
Epoch 40/67
 - 3s - loss: 0.3358 - accuracy: 0.8785
Epoch 41/67
 - 3s - loss: 0.3124 - accuracy: 0.8855
Epoch 42/67
 - 3s - loss: 0.3384 - accuracy: 0.8768
Epoch 43/67
 - 3s - loss: 0.3220 - accuracy: 0.8831
Epoch 44/67
 - 3s - loss: 0.3020 - accuracy: 0.8946
Epoch 45/67
 - 3s - loss: 0.3020 - accuracy: 0.8921
Epoch 46/67
 - 3s - loss: 0.2983 - accuracy: 0.8909
Epoch 47/67
 - 3s - loss: 0.2900 - accuracy: 0.8953
Epoch 48/67
 - 3s - loss: 0.2677 - accuracy: 0.9047
Epoch 49/67
 - 3s - loss: 0.2894 - accuracy: 0.8950
Epoch 50/67
 - 3s - loss: 0.2756 - accuracy: 0.9022
Epoch 51/67
 - 3s - loss: 0.2559 - accuracy: 0.9085
Epoch 52/67
 - 3s - loss: 0.2746 - accuracy: 0.9021
Epoch 53/67
 - 3s - loss

 - 3s - loss: 0.2021 - accuracy: 0.9296
Epoch 59/67
 - 2s - loss: 0.2799 - accuracy: 0.9001
Epoch 60/67
 - 2s - loss: 0.2320 - accuracy: 0.9189
Epoch 61/67
 - 2s - loss: 0.1914 - accuracy: 0.9357
Epoch 62/67
 - 2s - loss: 0.2119 - accuracy: 0.9271
Epoch 63/67
 - 2s - loss: 0.2293 - accuracy: 0.9185
Epoch 64/67
 - 2s - loss: 0.2059 - accuracy: 0.9293
Epoch 65/67
 - 2s - loss: 0.1781 - accuracy: 0.9393
Epoch 66/67
 - 2s - loss: 0.1800 - accuracy: 0.9386
Epoch 67/67
 - 2s - loss: 0.1780 - accuracy: 0.9413
Epoch 1/67
 - 3s - loss: 1.1660 - accuracy: 0.6092
Epoch 2/67
 - 3s - loss: 0.8895 - accuracy: 0.6889
Epoch 3/67
 - 3s - loss: 0.8093 - accuracy: 0.7210
Epoch 4/67
 - 2s - loss: 0.7410 - accuracy: 0.7490
Epoch 5/67
 - 2s - loss: 0.7140 - accuracy: 0.7560
Epoch 6/67
 - 2s - loss: 0.6737 - accuracy: 0.7675
Epoch 7/67
 - 2s - loss: 0.6438 - accuracy: 0.7782
Epoch 8/67
 - 2s - loss: 0.6454 - accuracy: 0.7692
Epoch 9/67
 - 2s - loss: 0.6099 - accuracy: 0.7862
Epoch 10/67
 - 2s - loss: 0.5850 

Epoch 10/30
 - 2s - loss: 0.6328 - accuracy: 0.7775
Epoch 11/30
 - 2s - loss: 0.6198 - accuracy: 0.7816
Epoch 12/30
 - 2s - loss: 0.6137 - accuracy: 0.7849
Epoch 13/30
 - 2s - loss: 0.6073 - accuracy: 0.7850
Epoch 14/30
 - 2s - loss: 0.6011 - accuracy: 0.7856
Epoch 15/30
 - 2s - loss: 0.5956 - accuracy: 0.7881
Epoch 16/30
 - 2s - loss: 0.5896 - accuracy: 0.7903
Epoch 17/30
 - 2s - loss: 0.5835 - accuracy: 0.7932
Epoch 18/30
 - 2s - loss: 0.5784 - accuracy: 0.7970
Epoch 19/30
 - 2s - loss: 0.5728 - accuracy: 0.7970
Epoch 20/30
 - 2s - loss: 0.5686 - accuracy: 0.7975
Epoch 21/30
 - 2s - loss: 0.5656 - accuracy: 0.7995
Epoch 22/30
 - 2s - loss: 0.5591 - accuracy: 0.7995
Epoch 23/30
 - 2s - loss: 0.5585 - accuracy: 0.7990
Epoch 24/30
 - 2s - loss: 0.5522 - accuracy: 0.8027
Epoch 25/30
 - 2s - loss: 0.5504 - accuracy: 0.8040
Epoch 26/30
 - 2s - loss: 0.5478 - accuracy: 0.8035
Epoch 27/30
 - 2s - loss: 0.5403 - accuracy: 0.8046
Epoch 28/30
 - 2s - loss: 0.5406 - accuracy: 0.8074
Epoch 29/30


...

Epoch 15/44
 - 3s - loss: 0.5954 - accuracy: 0.7862
Epoch 16/44
 - 3s - loss: 0.5728 - accuracy: 0.7957
Epoch 17/44
 - 3s - loss: 0.5681 - accuracy: 0.7971
Epoch 18/44
 - 4s - loss: 0.5607 - accuracy: 0.7980
Epoch 19/44
 - 3s - loss: 0.5497 - accuracy: 0.8014
Epoch 20/44
 - 3s - loss: 0.5393 - accuracy: 0.8035
Epoch 21/44
 - 4s - loss: 0.5395 - accuracy: 0.8035
Epoch 22/44
 - 4s - loss: 0.5270 - accuracy: 0.8089
Epoch 23/44
 - 4s - loss: 0.5218 - accuracy: 0.8112
Epoch 24/44
 - 4s - loss: 0.5083 - accuracy: 0.8125
Epoch 25/44
 - 4s - loss: 0.5062 - accuracy: 0.8159
Epoch 26/44
 - 4s - loss: 0.4972 - accuracy: 0.8193
Epoch 27/44
 - 4s - loss: 0.4871 - accuracy: 0.8242
Epoch 28/44
 - 4s - loss: 0.4842 - accuracy: 0.8263
Epoch 29/44
 - 4s - loss: 0.4890 - accuracy: 0.8221
Epoch 30/44
 - 4s - loss: 0.4725 - accuracy: 0.8261
Epoch 31/44
 - 4s - loss: 0.4657 - accuracy: 0.8296
Epoch 32/44
 - 4s - loss: 0.4610 - accuracy: 0.8309
Epoch 33/44
 - 4s - loss: 0.4627 - accuracy: 0.8325
Epoch 34/44


 - 4s - loss: 0.4239 - accuracy: 0.8415
Epoch 42/44
 - 4s - loss: 0.4060 - accuracy: 0.8527
Epoch 43/44
 - 4s - loss: 0.4133 - accuracy: 0.8458
Epoch 44/44
 - 4s - loss: 0.4026 - accuracy: 0.8521
| 11        | 0.8258    | 4.218     | 415.5     | 0.7479    | 0.2359    | 43.75     | 1.448     | 2.344     | 2.777     | 0.03051   | 98.84     | 0.1919    | 2.446     |
Epoch 1/27
 - 5s - loss: 1.2635 - accuracy: 0.6244
Epoch 2/27
 - 3s - loss: 1.1701 - accuracy: 0.6440
Epoch 3/27
 - 3s - loss: 1.1810 - accuracy: 0.6440
Epoch 4/27
 - 3s - loss: 1.1879 - accuracy: 0.6440
Epoch 5/27
 - 3s - loss: 1.1810 - accuracy: 0.6440
Epoch 6/27
 - 3s - loss: 1.1688 - accuracy: 0.6440
Epoch 7/27
 - 3s - loss: 1.1928 - accuracy: 0.6440
Epoch 8/27
 - 3s - loss: 1.1722 - accuracy: 0.6440
Epoch 9/27
 - 3s - loss: 1.1826 - accuracy: 0.6440
Epoch 10/27
 - 3s - loss: 1.1704 - accuracy: 0.6440
Epoch 11/27
 - 3s - loss: 1.1648 - accuracy: 0.6440
Epoch 12/27
 - 3s - loss: 1.1652 - accuracy: 0.6440
Epoch 13/27
 - 3s -

Epoch 33/91
 - 3s - loss: 0.7151 - accuracy: 0.7557
Epoch 34/91
 - 2s - loss: 0.7112 - accuracy: 0.7541
Epoch 35/91
 - 2s - loss: 0.7077 - accuracy: 0.7552
Epoch 36/91
 - 2s - loss: 0.7016 - accuracy: 0.7568
Epoch 37/91
 - 2s - loss: 0.6974 - accuracy: 0.7556
Epoch 38/91
 - 2s - loss: 0.6930 - accuracy: 0.7591
Epoch 39/91
 - 2s - loss: 0.6908 - accuracy: 0.7582
Epoch 40/91
 - 2s - loss: 0.6856 - accuracy: 0.7632
Epoch 41/91
 - 2s - loss: 0.6829 - accuracy: 0.7621
Epoch 42/91
 - 2s - loss: 0.6803 - accuracy: 0.7643
Epoch 43/91
 - 2s - loss: 0.6798 - accuracy: 0.7628
Epoch 44/91
 - 2s - loss: 0.6752 - accuracy: 0.7635
Epoch 45/91
 - 2s - loss: 0.6694 - accuracy: 0.7650
Epoch 46/91
 - 2s - loss: 0.6679 - accuracy: 0.7646
Epoch 47/91
 - 2s - loss: 0.6695 - accuracy: 0.7678
Epoch 48/91
 - 2s - loss: 0.6615 - accuracy: 0.7707
Epoch 49/91
 - 2s - loss: 0.6609 - accuracy: 0.7685
Epoch 50/91
 - 2s - loss: 0.6582 - accuracy: 0.7685
Epoch 51/91
 - 2s - loss: 0.6628 - accuracy: 0.7673
Epoch 52/91


Epoch 9/91
 - 3s - loss: 0.9715 - accuracy: 0.6532
Epoch 10/91
 - 2s - loss: 0.9533 - accuracy: 0.6591
Epoch 11/91
 - 3s - loss: 0.9368 - accuracy: 0.6605
Epoch 12/91
 - 3s - loss: 0.9204 - accuracy: 0.6655
Epoch 13/91
 - 3s - loss: 0.9024 - accuracy: 0.6710
Epoch 14/91
 - 3s - loss: 0.8818 - accuracy: 0.6743
Epoch 15/91
 - 3s - loss: 0.8548 - accuracy: 0.6913
Epoch 16/91
 - 2s - loss: 0.8322 - accuracy: 0.7003
Epoch 17/91
 - 3s - loss: 0.8120 - accuracy: 0.7126
Epoch 18/91
 - 3s - loss: 0.7897 - accuracy: 0.7244
Epoch 19/91
 - 3s - loss: 0.7751 - accuracy: 0.7330
Epoch 20/91
 - 3s - loss: 0.7616 - accuracy: 0.7364
Epoch 21/91
 - 3s - loss: 0.7504 - accuracy: 0.7402
Epoch 22/91
 - 2s - loss: 0.7389 - accuracy: 0.7443
Epoch 23/91
 - 3s - loss: 0.7323 - accuracy: 0.7476
Epoch 24/91
 - 3s - loss: 0.7222 - accuracy: 0.7507
Epoch 25/91
 - 2s - loss: 0.7125 - accuracy: 0.7516
Epoch 26/91
 - 3s - loss: 0.7059 - accuracy: 0.7550
Epoch 27/91
 - 2s - loss: 0.6985 - accuracy: 0.7594
Epoch 28/91
 

Epoch 76/91
 - 2s - loss: 0.5786 - accuracy: 0.8002
Epoch 77/91
 - 2s - loss: 0.5757 - accuracy: 0.8024
Epoch 78/91
 - 2s - loss: 0.5776 - accuracy: 0.8022
Epoch 79/91
 - 2s - loss: 0.5722 - accuracy: 0.8057
Epoch 80/91
 - 2s - loss: 0.5717 - accuracy: 0.8047
Epoch 81/91
 - 2s - loss: 0.5666 - accuracy: 0.8044
Epoch 82/91
 - 2s - loss: 0.5634 - accuracy: 0.8068
Epoch 83/91
 - 2s - loss: 0.5680 - accuracy: 0.8045
Epoch 84/91
 - 2s - loss: 0.5652 - accuracy: 0.8044
Epoch 85/91
 - 2s - loss: 0.5592 - accuracy: 0.8073
Epoch 86/91
 - 3s - loss: 0.5616 - accuracy: 0.8073
Epoch 87/91
 - 2s - loss: 0.5610 - accuracy: 0.8057
Epoch 88/91
 - 3s - loss: 0.5515 - accuracy: 0.8106
Epoch 89/91
 - 2s - loss: 0.5512 - accuracy: 0.8113
Epoch 90/91
 - 3s - loss: 0.5494 - accuracy: 0.8084
Epoch 91/91
 - 2s - loss: 0.5521 - accuracy: 0.8097
Epoch 1/91
 - 5s - loss: 1.3024 - accuracy: 0.6240
Epoch 2/91
 - 3s - loss: 1.1564 - accuracy: 0.6439
Epoch 3/91
 - 3s - loss: 1.1396 - accuracy: 0.6439
Epoch 4/91
 - 3

In [47]:
optimum = nn_opt.max['params']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]
optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Adam']
optimizerD = {
    'Adam': Adam(),
    'SGD': SGD(),
    'RMSprop': RMSprop(),
    'Adadelta': Adadelta(),
    'Adagrad': Adagrad(),
    'Adamax': Adamax(),
    'Nadam': Nadam()
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'activation': 'softsign',
 'batch_size': 460,
 'dropout': 0.7296061783380641,
 'dropout_rate': 0.19126724140656393,
 'epochs': 91,
 'kernel': 1.9444298503238986,
 'layers1': 1,
 'layers2': 2,
 'learning_rate': 0.7631771981307285,
 'neurons': 61,
 'normalization': 0.770967179954561,
 'optimizer': <tensorflow.python.keras.optimizers.Adadelta at 0x1d0429517b8>}

In [56]:
epochs = 91
batch_size = 460

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 #_count_classes(Y_train)
layers1 = 1
layers2 = 2
activation = 'softsign'
kernel = 2
neurons = 61
normalization = 0.770967179954561
dropout = 0.7296
dropout_rate = 0.1912
optimizer = 'Adadelta'

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))
if normalization > 0.5:
    model.add(BatchNormalization())
for i in range(layers1):
    model.add(Dense(neurons, activation=activation))
if dropout > 0.5:
    model.add(Dropout(dropout_rate, seed=123))
for i in range(layers2):
    model.add(Dense(neurons, activation=activation))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) #softmax sigmoid
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #binary_crossentropy

In [57]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/91
 - 5s - loss: 1.2213 - accuracy: 0.6488
Epoch 2/91
 - 3s - loss: 0.9020 - accuracy: 0.7079
Epoch 3/91
 - 3s - loss: 0.8289 - accuracy: 0.7222
Epoch 4/91
 - 3s - loss: 0.7702 - accuracy: 0.7393
Epoch 5/91
 - 3s - loss: 0.7343 - accuracy: 0.7485
Epoch 6/91
 - 4s - loss: 0.7070 - accuracy: 0.7578
Epoch 7/91
 - 3s - loss: 0.6835 - accuracy: 0.7655
Epoch 8/91
 - 4s - loss: 0.6670 - accuracy: 0.7691
Epoch 9/91
 - 3s - loss: 0.6576 - accuracy: 0.7693
Epoch 10/91
 - 3s - loss: 0.6430 - accuracy: 0.7755
Epoch 11/91
 - 4s - loss: 0.6286 - accuracy: 0.7824
Epoch 12/91
 - 3s - loss: 0.6277 - accuracy: 0.7794
Epoch 13/91
 - 4s - loss: 0.6200 - accuracy: 0.7804
Epoch 14/91
 - 3s - loss: 0.6072 - accuracy: 0.7831
Epoch 15/91
 - 3s - loss: 0.5960 - accuracy: 0.7879
Epoch 16/91
 - 3s - loss: 0.5965 - accuracy: 0.7851
Epoch 17/91
 - 3s - loss: 0.5804 - accuracy: 0.7931
Epoch 18/91
 - 3s - loss: 0.5798 - accuracy: 0.7936
Epoch 19/91
 - 3s - loss: 0.5740 - accuracy: 0.7946
Epoch 20/91
 - 3s - l

In [69]:
stations = {0: 'BASEL', 1: 'BELGRADE', 2: 'BUDAPEST', 3: 'DEBILT', 4: 'DUSSELDORF', 5: 'HEATHROW', 6: 'KASSEL', 7: 'LJUBLJANA', 8: 'MAASTRICHT', 9: 'MADRID', 10: 'MUNCHENB', 11: 'OSLO', 12: 'SONNBLICK', 13: 'STOCKHOLM', 14: 'VALENTIA'}

def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([stations[y] for y in Y_true])
    Y_pred = pd.Series([stations[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [75]:
acc = (np.sum(y_test == [y for y in np.argmax(model.predict(X_test), axis=1)]) / len(y_test)) * 100.0 

In [76]:
# Evaluate
print("ACCURACY : " + str(acc))
print(confusion_matrix(y_test, model.predict(X_test)))

ACCURACY : 82.76402927849425
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        3545       102        15       3           2         0       0   
BELGRADE      306       781         3       0           0         0       0   
BUDAPEST       60        66        88       0           0         0       0   
DEBILT         33        14        20      15           0         0       0   
DUSSELDORF     15         4         6       2           2         0       0   
HEATHROW       21        12         8       3           3        27       0   
KASSEL          6         3         1       0           0         0       1   
LJUBLJANA      21        10         6       0           0         1       1   
MAASTRICHT      8         0         0       1           0         0       0   
MADRID         84        55        27       2           1        13       0   
MUNCHENB        8      